# Etape 3 - Nettoyage avance Pandas

**Objectif** : Preparer les donnees meteo et fusionner avec les donnees de pollution

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuration affichage
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Chemins
DATA_DIR = "../data"
OUTPUT_DIR = "../output"
PARQUET_DIR = "../output/air_quality_clean"

## 3.1 Chargement des donnees meteo brutes

In [2]:
# Charger les donnees meteo
df_weather_raw = pd.read_csv(f"{DATA_DIR}/weather_raw.csv")

print(f"Shape: {df_weather_raw.shape}")
print(f"\nColonnes: {df_weather_raw.columns.tolist()}")
df_weather_raw.head(10)

Shape: (42172, 7)

Colonnes: ['city', 'timestamp', 'temperature_c', 'humidity_pct', 'wind_speed_kmh', 'precipitation_mm', 'weather_condition']


,city,timestamp,temperature_c,humidity_pct,wind_speed_kmh,precipitation_mm,weather_condition
0,Grenoble,2024-03-12 09:00:00,14.8,74.6,24.8,2.4,pluvieux
1,Strasbourg,05/04/2024 21:00,10.8,78.4,5.7,0.0,neigeux
2,Marseille,28/01/2024 23:00,13.3,60.4,45.7,3.5,pluvieux
3,Bordeaux,29/01/2024 12:00,1.3,84.1,43.3,0.0,brumeux
4,Marseille,2024-01-15 00:00:00,13.3,87.4,21.7,6.0,orageux
5,Lille,2024-05-14 14:00:00,15.7,45.4,0.0,0.0,brumeux
6,Paris,2024-02-12 22:00:00,1.5,93.8,29.6,0.0,brumeux
7,Nantes,04/17/2024 14:00:00,19.2,51.2,49.4,0.0,pluvieux
8,Bordeaux,04/25/2024 09:00:00,"19,9",43.6,22.9,0.7,pluvieux
9,Toulouse,01/06/2024 13:00,35.7,61.7,29.2,6.2,pluvieux


In [3]:
# Info sur les types
df_weather_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42172 entries, 0 to 42171
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   city               42172 non-null  object 
 1   timestamp          42172 non-null  object 
 2   temperature_c      41724 non-null  object 
 3   humidity_pct       41760 non-null  float64
 4   wind_speed_kmh     42172 non-null  float64
 5   precipitation_mm   42172 non-null  float64
 6   weather_condition  41310 non-null  object 
dtypes: float64(3), object(4)
memory usage: 2.3+ MB


## 3.2 Diagnostic de la qualite des donnees

In [4]:
def quality_report(df, name="DataFrame"):
    print(f"RAPPORT QUALITE - {name}")
    print(f"Lignes: {len(df):,}")
    print(f"Colonnes: {len(df.columns)}")
    
    report = []
    for col in df.columns:
        total = len(df)
        missing = df[col].isna().sum() + (df[col] == '').sum() if df[col].dtype == 'object' else df[col].isna().sum()
        completude = (1 - missing / total) * 100
        unique = df[col].nunique()
        dtype = df[col].dtype
        
        report.append({
            'Colonne': col,
            'Type': str(dtype),
            'Manquants': missing,
            'Completude %': round(completude, 2),
            'Uniques': unique
        })
    
    return pd.DataFrame(report)

quality_report(df_weather_raw, "Weather Raw")

RAPPORT QUALITE - Weather Raw
Lignes: 42,172
Colonnes: 7


,Colonne,Type,Manquants,Completude %,Uniques
0,city,object,0,100.00,10
1,timestamp,object,0,100.00,16406
2,temperature_c,object,448,98.94,816
3,humidity_pct,float64,412,99.02,836
4,wind_speed_kmh,float64,0,100.00,501
5,precipitation_mm,float64,0,100.00,101
6,weather_condition,object,862,97.96,6


In [5]:
# Examiner les valeurs problematiques dans temperature_c
print("Valeurs uniques non numeriques dans temperature_c:")
temp_non_numeric = df_weather_raw[
    ~df_weather_raw['temperature_c'].astype(str).str.match(r'^-?[0-9]+[.,]?[0-9]*$', na=False)
]['temperature_c'].unique()
print(temp_non_numeric)

Valeurs uniques non numeriques dans temperature_c:
[nan]


In [6]:
# Valeurs aberrantes de temperature
df_temp = df_weather_raw.copy()
df_temp['temperature_c'] = pd.to_numeric(
    df_temp['temperature_c'].astype(str).str.replace(',', '.'), 
    errors='coerce'
)

print("Distribution temperature:")
print(df_temp['temperature_c'].describe())

print(f"\nTemperatures < -40: {(df_temp['temperature_c'] < -40).sum()}")
print(f"Temperatures > 50: {(df_temp['temperature_c'] > 50).sum()}")

Distribution temperature:
count    41724.000000
mean        13.995688
std         11.963235
min        -80.000000
25%          7.900000
50%         13.500000
75%         20.100000
max        100.000000
Name: temperature_c, dtype: float64

Temperatures < -40: 165
Temperatures > 50: 290


In [7]:
# Humidite hors bornes
df_temp['humidity_pct'] = pd.to_numeric(df_temp['humidity_pct'], errors='coerce')

print("\nDistribution humidite:")
print(df_temp['humidity_pct'].describe())

print(f"\nHumidite > 100%: {(df_temp['humidity_pct'] > 100).sum()}")
print(f"Humidite < 0%: {(df_temp['humidity_pct'] < 0).sum()}")


Distribution humidite:
count    41760.000000
mean        68.171889
std         16.974696
min         40.000000
25%         54.000000
50%         67.900000
75%         81.700000
max        150.000000
Name: humidity_pct, dtype: float64

Humidite > 100%: 454
Humidite < 0%: 0


## 3.3 Nettoyage des donnees meteo

In [8]:
def parse_timestamp(ts):
    """Parse les timestamps multi-formats."""
    if pd.isna(ts):
        return pd.NaT
    
    formats = [
        "%Y-%m-%d %H:%M:%S",
        "%d/%m/%Y %H:%M",
        "%m/%d/%Y %H:%M:%S",
        "%Y-%m-%dT%H:%M:%S",
    ]
    
    for fmt in formats:
        try:
            return datetime.strptime(str(ts), fmt)
        except ValueError:
            continue
    
    return pd.NaT

df_weather = df_weather_raw.copy()

# 1. Parser les timestamps
print("[1/5] Parsing des timestamps...")
df_weather['timestamp'] = df_weather['timestamp'].apply(parse_timestamp)
invalid_ts = df_weather['timestamp'].isna().sum()
print(f"  Timestamps invalides: {invalid_ts}")

# Supprimer les lignes sans timestamp valide
df_weather = df_weather.dropna(subset=['timestamp'])

[1/5] Parsing des timestamps...
  Timestamps invalides: 0


In [9]:
# 2. Convertir les colonnes numeriques
print("\n[2/5] Conversion des colonnes numeriques...")

# Temperature (remplacer virgule par point)
df_weather['temperature_c'] = pd.to_numeric(
    df_weather['temperature_c'].astype(str).str.replace(',', '.'),
    errors='coerce'
)

# Autres colonnes
df_weather['humidity_pct'] = pd.to_numeric(df_weather['humidity_pct'], errors='coerce')
df_weather['wind_speed_kmh'] = pd.to_numeric(df_weather['wind_speed_kmh'], errors='coerce')
df_weather['precipitation_mm'] = pd.to_numeric(df_weather['precipitation_mm'], errors='coerce')

print("  Conversions effectuees.")


[2/5] Conversion des colonnes numeriques...
  Conversions effectuees.


In [10]:
# 3. Corriger les valeurs aberrantes
print("\n[3/5] Correction des valeurs aberrantes...")

# Temperatures hors [-40, 50] -> NaN
temp_outliers = ((df_weather['temperature_c'] < -40) | (df_weather['temperature_c'] > 50)).sum()
df_weather.loc[
    (df_weather['temperature_c'] < -40) | (df_weather['temperature_c'] > 50),
    'temperature_c'
] = np.nan
print(f"  Temperatures aberrantes -> NaN: {temp_outliers}")

# Humidite hors [0, 100] -> clipper
humidity_outliers = ((df_weather['humidity_pct'] < 0) | (df_weather['humidity_pct'] > 100)).sum()
df_weather['humidity_pct'] = df_weather['humidity_pct'].clip(0, 100)
print(f"  Humidite clippee [0, 100]: {humidity_outliers}")


[3/5] Correction des valeurs aberrantes...
  Temperatures aberrantes -> NaN: 455
  Humidite clippee [0, 100]: 454


In [11]:
# 4. Traiter les valeurs manquantes
print("\n[4/5] Traitement des valeurs manquantes...")

# Trier par ville et timestamp pour l'interpolation
df_weather = df_weather.sort_values(['city', 'timestamp'])

# Interpolation lineaire pour temperature et humidite (par ville)
for col in ['temperature_c', 'humidity_pct', 'wind_speed_kmh', 'precipitation_mm']:
    before_na = df_weather[col].isna().sum()
    df_weather[col] = df_weather.groupby('city')[col].transform(
        lambda x: x.interpolate(method='linear', limit_direction='both')
    )
    after_na = df_weather[col].isna().sum()
    print(f"  {col}: {before_na} -> {after_na} NaN (interpoles: {before_na - after_na})")

# Forward fill pour weather_condition
before_na = (df_weather['weather_condition'].isna() | (df_weather['weather_condition'] == '')).sum()
df_weather['weather_condition'] = df_weather['weather_condition'].replace('', np.nan)
df_weather['weather_condition'] = df_weather.groupby('city')['weather_condition'].transform(
    lambda x: x.ffill().bfill()
)
after_na = df_weather['weather_condition'].isna().sum()
print(f"  weather_condition: {before_na} -> {after_na} NaN (forward filled)")


[4/5] Traitement des valeurs manquantes...
  temperature_c: 903 -> 0 NaN (interpoles: 903)
  humidity_pct: 412 -> 0 NaN (interpoles: 412)
  wind_speed_kmh: 0 -> 0 NaN (interpoles: 0)
  precipitation_mm: 0 -> 0 NaN (interpoles: 0)
  weather_condition: 862 -> 0 NaN (forward filled)


In [12]:
# 5. Ajouter des colonnes temporelles
print("\n[5/5] Ajout des colonnes temporelles...")

df_weather['date'] = df_weather['timestamp'].dt.date
df_weather['hour'] = df_weather['timestamp'].dt.hour
df_weather['day_of_week'] = df_weather['timestamp'].dt.dayofweek
df_weather['month'] = df_weather['timestamp'].dt.month
df_weather['season'] = df_weather['month'].map({
    12: 'Hiver', 1: 'Hiver', 2: 'Hiver',
    3: 'Printemps', 4: 'Printemps', 5: 'Printemps',
    6: 'Ete', 7: 'Ete', 8: 'Ete',
    9: 'Automne', 10: 'Automne', 11: 'Automne'
})

print("  Colonnes ajoutees: date, hour, day_of_week, month, season")


[5/5] Ajout des colonnes temporelles...
  Colonnes ajoutees: date, hour, day_of_week, month, season


In [13]:
# Rapport apres nettoyage
quality_report(df_weather, "Weather Clean")

RAPPORT QUALITE - Weather Clean
Lignes: 42,172
Colonnes: 12


,Colonne,Type,Manquants,Completude %,Uniques
0,city,object,0,100.0,10
1,timestamp,datetime64[ns],0,100.0,4368
2,temperature_c,float64,0,100.0,783
3,humidity_pct,float64,0,100.0,750
4,wind_speed_kmh,float64,0,100.0,501
5,precipitation_mm,float64,0,100.0,101
6,weather_condition,object,0,100.0,6
7,date,object,0,100.0,182
8,hour,int32,0,100.0,24
9,day_of_week,int32,0,100.0,7


## 3.4 Chargement des donnees de pollution nettoyees

In [14]:
# Charger les donnees Parquet (produites par l'etape 2)
import os

if os.path.exists(PARQUET_DIR):
    df_pollution = pd.read_parquet(PARQUET_DIR)
    print(f"Donnees pollution chargees: {len(df_pollution):,} lignes")
    df_pollution.head()
else:
    # Si les donnees Parquet n'existent pas, charger le CSV brut et nettoyer
    print("Parquet non trouve, chargement du CSV brut...")
    df_air_raw = pd.read_csv(f"{DATA_DIR}/air_quality_raw.csv")
    df_stations = pd.read_csv(f"{DATA_DIR}/stations.csv")
    
    # Nettoyage simplifie
    df_pollution = df_air_raw.copy()
    df_pollution['timestamp'] = df_pollution['timestamp'].apply(parse_timestamp)
    df_pollution = df_pollution.dropna(subset=['timestamp'])
    df_pollution['value'] = pd.to_numeric(
        df_pollution['value'].astype(str).str.replace(',', '.'),
        errors='coerce'
    )
    df_pollution = df_pollution.dropna(subset=['value'])
    df_pollution = df_pollution[(df_pollution['value'] >= 0) & (df_pollution['value'] <= 1000)]
    df_pollution = df_pollution.drop_duplicates(subset=['station_id', 'timestamp', 'pollutant'])
    
    # Joindre avec stations
    df_pollution = df_pollution.merge(
        df_stations[['station_id', 'city', 'station_type']],
        on='station_id',
        how='left'
    )
    
    # Renommer pour coherence
    df_pollution = df_pollution.rename(columns={'value': 'value_mean'})
    
    print(f"Donnees pollution nettoyees: {len(df_pollution):,} lignes")

Donnees pollution chargees: 876,800 lignes


In [15]:
# Apercu des donnees pollution
df_pollution.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 876800 entries, 0 to 876799
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   station_id         876800 non-null  object  
 1   pollutant          876800 non-null  object  
 2   unit               876800 non-null  object  
 3   hour               876800 non-null  int32   
 4   year               876800 non-null  int32   
 5   month              876800 non-null  int32   
 6   value_mean         876800 non-null  float64 
 7   value_min          876800 non-null  float64 
 8   value_max          876800 non-null  float64 
 9   measurement_count  876800 non-null  int64   
 10  station_name       876800 non-null  object  
 11  city               876800 non-null  object  
 12  station_type       876800 non-null  object  
 13  date               876800 non-null  category
dtypes: category(1), float64(3), int32(3), int64(1), object(6)
memory usage: 78.6+ MB


## 3.5 Fusion pollution/meteo

In [16]:
# Preparer les donnees pour la fusion

# Verifier si les donnees de pollution sont vides
if len(df_pollution) == 0:
    print("ATTENTION: Les donnees de pollution sont vides!")
    print("Veuillez d'abord executer le notebook 02 pour generer les donnees Parquet.")
    # Creer un DataFrame vide avec les bonnes colonnes pour eviter les erreurs
    df_pollution['datetime_hour'] = pd.to_datetime([])
else:
    # Pour la pollution: creer une colonne datetime arrondie a l'heure
    if 'date' in df_pollution.columns and 'hour' in df_pollution.columns:
        df_pollution['datetime_hour'] = pd.to_datetime(
            df_pollution['date'].astype(str) + ' ' + df_pollution['hour'].astype(str) + ':00:00'
        )
    elif 'timestamp_parsed' in df_pollution.columns:
        df_pollution['datetime_hour'] = df_pollution['timestamp_parsed'].dt.floor('H')
    elif 'timestamp' in df_pollution.columns:
        df_pollution['datetime_hour'] = df_pollution['timestamp'].dt.floor('H')
    else:
        print("ERREUR: Aucune colonne temporelle trouvee dans df_pollution")
        df_pollution['datetime_hour'] = pd.to_datetime([])

# Pour la meteo: arrondir a l'heure
df_weather['datetime_hour'] = df_weather['timestamp'].dt.floor('H')

if len(df_pollution) > 0:
    print(f"Pollution: {df_pollution['datetime_hour'].min()} -> {df_pollution['datetime_hour'].max()}")
print(f"Meteo: {df_weather['datetime_hour'].min()} -> {df_weather['datetime_hour'].max()}")

Pollution: 2024-01-01 00:00:00 -> 2024-06-30 23:00:00
Meteo: 2024-01-01 00:00:00 -> 2024-06-30 23:00:00


In [17]:
# Agreger la meteo par ville et heure (au cas ou il y aurait des doublons)
df_weather_hourly = df_weather.groupby(['city', 'datetime_hour']).agg({
    'temperature_c': 'mean',
    'humidity_pct': 'mean',
    'wind_speed_kmh': 'mean',
    'precipitation_mm': 'sum',
    'weather_condition': 'first',
    'season': 'first'
}).reset_index()

print(f"Meteo agregee: {len(df_weather_hourly):,} lignes")

Meteo agregee: 42,172 lignes


In [18]:
# Fusion sur ville et datetime_hour
if len(df_pollution) == 0:
    print("ERREUR: Impossible de fusionner - les donnees de pollution sont vides!")
    print("Veuillez d'abord executer le notebook 02_nettoyage_spark.ipynb")
    df_merged = pd.DataFrame()
else:
    df_merged = df_pollution.merge(
        df_weather_hourly,
        on=['city', 'datetime_hour'],
        how='left'
    )

    print(f"\nResultat de la fusion:")
    print(f"  Lignes pollution: {len(df_pollution):,}")
    print(f"  Lignes apres fusion: {len(df_merged):,}")
    print(f"  Lignes avec meteo: {df_merged['temperature_c'].notna().sum():,}")
    print(f"  Lignes sans meteo: {df_merged['temperature_c'].isna().sum():,}")


Resultat de la fusion:
  Lignes pollution: 876,800
  Lignes apres fusion: 876,800
  Lignes avec meteo: 845,519
  Lignes sans meteo: 31,281


In [19]:
# Apercu du dataset fusionne
df_merged.head(10)

,station_id,pollutant,unit,hour,year,month,value_mean,value_min,value_max,measurement_count,station_name,city,station_type,date,datetime_hour,temperature_c,humidity_pct,wind_speed_kmh,precipitation_mm,weather_condition,season
0,ST0001,CO,mg/m3,1,2024,1,0.38,0.38,0.38,1,Paris-urbaine-1,Paris,urbaine,2024-01-01,2024-01-01 01:00:00,-1.0,67.1,48.9,9.1,pluvieux,Hiver
1,ST0002,SO2,ug/m3,5,2024,1,3.75,3.75,3.75,1,Paris-periurbaine-2,Paris,periurbaine,2024-01-01,2024-01-01 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2,ST0003,SO2,ug/m3,22,2024,1,9.44,9.44,9.44,1,Paris-industrielle-3,Paris,industrielle,2024-01-01,2024-01-01 22:00:00,3.4,61.7,46.2,0.0,orageux,Hiver
3,ST0004,PM10,ug/m3,7,2024,1,58.05,58.05,58.05,1,Lyon-urbaine-1,Lyon,urbaine,2024-01-01,2024-01-01 07:00:00,0.8,43.0,18.7,0.0,pluvieux,Hiver
4,ST0004,SO2,ug/m3,10,2024,1,5.50,5.50,5.50,1,Lyon-urbaine-1,Lyon,urbaine,2024-01-01,2024-01-01 10:00:00,2.3,58.8,21.7,3.0,pluvieux,Hiver
5,ST0005,SO2,ug/m3,1,2024,1,4.79,4.79,4.79,1,Lyon-periurbaine-2,Lyon,periurbaine,2024-01-01,2024-01-01 01:00:00,-1.4,89.7,16.8,0.0,brumeux,Hiver
6,ST0005,SO2,ug/m3,8,2024,1,10.78,10.78,10.78,1,Lyon-periurbaine-2,Lyon,periurbaine,2024-01-01,2024-01-01 08:00:00,12.2,44.3,14.5,0.0,neigeux,Hiver
7,ST0005,CO,mg/m3,12,2024,1,0.47,0.47,0.47,1,Lyon-periurbaine-2,Lyon,periurbaine,2024-01-01,2024-01-01 12:00:00,7.4,40.6,22.8,0.0,neigeux,Hiver
8,ST0005,O3,ug/m3,12,2024,1,54.14,54.14,54.14,1,Lyon-periurbaine-2,Lyon,periurbaine,2024-01-01,2024-01-01 12:00:00,7.4,40.6,22.8,0.0,neigeux,Hiver
9,ST0005,CO,mg/m3,13,2024,1,1.00,1.00,1.00,1,Lyon-periurbaine-2,Lyon,periurbaine,2024-01-01,2024-01-01 13:00:00,-2.2,48.5,39.4,3.2,pluvieux,Hiver


## 3.6 Rapport de nettoyage et sauvegarde

In [20]:
# Rapport final
print("RAPPORT DE NETTOYAGE - DONNEES METEO")
print(f"Lignes brutes: {len(df_weather_raw):,}")
print(f"Lignes apres nettoyage: {len(df_weather):,}")
print(f"Lignes supprimees: {len(df_weather_raw) - len(df_weather):,}")
print("\nCompletude finale:")
for col in ['temperature_c', 'humidity_pct', 'wind_speed_kmh', 'precipitation_mm', 'weather_condition']:
    completude = (1 - df_weather[col].isna().sum() / len(df_weather)) * 100
    print(f"  {col}: {completude:.2f}%")

RAPPORT DE NETTOYAGE - DONNEES METEO
Lignes brutes: 42,172
Lignes apres nettoyage: 42,172
Lignes supprimees: 0

Completude finale:
  temperature_c: 100.00%
  humidity_pct: 100.00%
  wind_speed_kmh: 100.00%
  precipitation_mm: 100.00%
  weather_condition: 100.00%


In [ ]:
# Sauvegarder le dataset fusionne
import os
os.makedirs(OUTPUT_DIR, exist_ok=True)

output_path = f"{OUTPUT_DIR}/pollution_meteo_clean.csv"
df_merged.to_csv(output_path, index=False)
print(f"Dataset fusionne sauvegarde: {output_path}")
print(f"Taille: {len(df_merged):,} lignes x {len(df_merged.columns)} colonnes")

In [ ]:
# Sauvegarder aussi en Parquet pour de meilleures performances
parquet_path = f"{OUTPUT_DIR}/pollution_meteo_clean.parquet"
df_merged.to_parquet(parquet_path, index=False)
print(f"Dataset Parquet sauvegarde: {parquet_path}")

Dataset Parquet sauvegarde: ../output/pollution_meteo_clean.parquet


In [ ]:
# Resume des colonnes du dataset final
print("\nColonnes du dataset final:")
df_merged.info()


Colonnes du dataset final:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame
